# Genetic Algorithm in Tensorflow

Let's create a genetic algorithm in tensorflow. Why use tensorflow? Because dataflow programming is magical! And tensorflow provides a bunch of nice things on top of it's dataflow execution engine. Like viusalization. And [optimizations](https://stackoverflow.com/questions/42628143/does-tensorflow-simplify-a-computational-graph)! It's a whole new world out there!

Ok, so let's start with a really simle GA. Evolve a length 100 array of all 1s. We have a population size of 100, and we start with random arrays of 1s and 0s. Mutate chooses one random index of the array and switches it. We use one point crossover, so an index $i$ is chosen between 0 and 99 and we select the parts of the inital array $0...i$ and the other array $i..99$. For our error, we use the number of 0s in the array. To select a parent, we choose it inversely proportional to it's error. We do 90% crossover and 10% mutate.

First let's write this in regular python so we know it works.

We also have a choice here about how to represent the current population in memory. For each individual in the population, it has at least a genome and an error. Assuming we are only holding the current generation's population in memory, we could either represent it as a list of individuals, each with an error and genome, or two lists of genomes and errors. Do we want a columbnar or row based representation? Let's think about what operations we are doing on the population. We want to be able to get a parent from the generation and also check if any individual has succeeded in it. Our method of parent selection is to select invesely proportional to the errors, so it is helpful to able to sum over errors. So I choice a columnar representation here.


In [2]:
import numpy as np

In [66]:
def new_genome():
    """
    Creates a random genome.
    
    A genome is a length 100 array of 1s and 0s
    """
    return np.random.randint(0, 2, 100)

assert(len(new_genome()) == 100)
assert(1 in new_genome())
assert(0 in new_genome())
assert(2 not in new_genome())

In [68]:
# the best possible genome, our solution we are looking for
genome_1s = np.ones(100)
# the worst genome
genome_0s = np.zeros(100)

In [69]:
def compute_error(genome):
    """
    Returns the error for a genome. This is the number 0s in it.
    """
    return 100 - np.sum(genome)

assert compute_error(genome_1s) == 0
assert compute_error(genome_0s) == 100

In [71]:
def mutate(genome):
    """
    Returns a new genome with one item in it randomly changed to a 1 or 0
    """
    index_to_change = np.random.randint(0, 100)
    # copy so we don't modify the original
    new_genome = np.copy(genome)
    new_genome[index_to_change] = np.random.randint(0, 2)
    return new_genome

assert error(mutate(genome_1s)) in [0, 1]
assert error(mutate(genome_0s)) in [100, 99]

In [74]:
def _crossover(genome_a, genome_b, point):
    return np.concatenate((genome_a[:point],  genome_b[point:]))

assert np.array_equal(_crossover(genome_1s, genome_0s, 0), genome_0s)
assert np.array_equal(_crossover(genome_1s, genome_0s, 100), genome_1s)
assert error(_crossover(genome_1s, genome_0s, 50)) == 50

def crossover(genome_a, genome_b):
    """
    Returns a new genome from two others, choosing a random pivot point
    and the part of genome_a up to that, with the part of genome_b from
    that point on.
    """
    crossover_point = np.random.randint(0, 101)
    return _crossover(genome_a, genome_b, crossover_point)
    

In [122]:
def errors_from_genomes(genomes):
    return np.array([compute_error(genome) for genome in genomes])

def population_from_genomes(genomes):
    return {"genomes": genomes, "errors": errors_from_genomes(genomes)}

def initial_population():
    """
    Returns an initial population of individuals. A population has
    two arrays, genomes and error. Each of length 100 so that that
    errors[i] is the error for genome[i].
    """
    genomes = np.array([new_genome() for _ in range(100)])
    return population_from_genomes(genomes)

assert len(initial_population()['genomes']) == 100

In [151]:
def select_parent(population):
    """
    Selects a parent from the population, with probability inversly
    proportional to it's error
    """
    unnormalized_ps = 1 / population['errors']
    ps = unnormalized_ps / np.sum(unnormalized_ps)
    index =  np.random.choice(100, p=ps)
    return population['genomes'][index]

very_bad = np.zeros(100)
very_good = np.ones(100)
very_good[0] = 0
genomes = np.concatenate((np.tile(very_bad, [50, 1]), np.tile(very_good, [50, 1])))
population = population_from_genomes(genomes)
assert compute_error(select_parent(population)) == compute_error(very_good)

In [152]:
def create_child(population):
    should_mutate = np.random.choice([True, False], p=[0.1, 0.9])
    if should_mutate:
        return mutate(select_parent(population))
    return crossover(select_parent(population), select_parent(population))

In [155]:
assert len(create_child(initial_population())) == 100

In [157]:
def create_children(population):
    return np.array([create_child(population) for _ in range(100)])

assert len(create_children(initial_population())) == 100

In [160]:
def have_finished(population):
    return np.any(population['errors'] == 0)

assert not have_finished(initial_population())

In [163]:
def best_error(population):
    return np.min(population['errors'])

In [164]:
def run():
    population = initial_population()
    while not have_finished(population):
        print(best_error(population))
        new_genomes = create_children(population)
        population = population_from_genomes(new_genomes)
    return population

In [165]:
run()

38
40
37
38
37
36
36
36
34
36
34
34
34
36
37
35
35
35
34
31
34
32
27
29
32
32
32
30
30
30
29
29
25
26
27
28
27
29
26
27
25
27
27
26
26
25
24
22
22
24
24
23
22
22
19
21
21
22
22
21
21
20
18
17
15
15
15
15
15
17
17
14
18
16
16
17
17
16
16
16
14
15
15
15
16
16
15
16
15
15
16
15
15
14
14
15
15
15
12
15
13
14
15
15
14
14
14
14
14
15
14
14
12
13
12
12
13
12
13
13
13
13
13
12
12
12
12
12
12
12
11
12
12
12
12
12
11
11
11
12
11
11
11
11
11
11
11
10
10
11
10
11
10
10
10
10
10
10
10
10
10
10
9
10
10
9
9
9
10
10
10
10
10
10
9
10
9
9
9
9
9
9
8
8
8
9
9
9
8
9
9
9
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
8
7
7
7
7
7
7
7
8
7
7
7
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
7
7
7
7
7
7
7
7
7
7
6
6
6
6
6
7
7
6
6
6
6
6
7
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1


{'errors': array([2, 1, 1, 1, 1, 1, 1, 3, 1, 3, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 3, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1,
        1, 2, 2, 2, 2, 2, 3, 1]), 'genomes': array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ..., 
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]])}

Eh! It works! 